In [2]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np

In [3]:
# Define the geographic boundary of the campus
place_name = "Georgia Institute of Technology, Atlanta, GA, USA"

# Load the walkable street network
G = ox.graph_from_place(place_name, network_type="walk")

In [13]:
# Student Living Centers (need actual data)... thanks tho chatgpt!
student_centroids = gpd.GeoDataFrame({
    "name": ["Dorm A", "Dorm B", "Apartment C"],
    "population": [500, 300, 700],
    "geometry": gpd.points_from_xy([-84.3963, -84.3951, -84.3890], [33.7756, 33.7742, 33.7725])
})

print(student_centroids.head())

          name  population                    geometry
0       Dorm A         500  POINT (-84.39630 33.77560)
1       Dorm B         300  POINT (-84.39510 33.77420)
2  Apartment C         700  POINT (-84.38900 33.77250)


In [15]:

# Existing Bike Shops (Replace with actual data)
existing_shops = gpd.GeoDataFrame({
    "name": ["Existing Shop 1", "Existing Shop 2"],
    "geometry": gpd.points_from_xy([-84.3915, -84.3885], [33.7760, 33.7745])
})

print(existing_shops.head())

              name                    geometry
0  Existing Shop 1  POINT (-84.39150 33.77600)
1  Existing Shop 2  POINT (-84.38850 33.77450)


In [16]:
# Candidate Bike Shops (Replace with actual data)
new_shops = gpd.GeoDataFrame({
    "name": ["New Shop 1", "New Shop 2", "New Shop 3"],
    "geometry": gpd.points_from_xy([-84.3920, -84.3900, -84.3880], [33.7750, 33.7730, 33.7710])
})

print(new_shops.head())


         name                    geometry
0  New Shop 1  POINT (-84.39200 33.77500)
1  New Shop 2  POINT (-84.39000 33.77300)
2  New Shop 3  POINT (-84.38800 33.77100)


In [17]:
# Combine all bike shops (existing + new)
all_shops = pd.concat([existing_shops, new_shops], ignore_index=True)

# Find nearest nodes in the street network
student_centroids["node"] = student_centroids.geometry.apply(lambda point: ox.nearest_nodes(G, point.x, point.y))
all_shops["node"] = all_shops.geometry.apply(lambda point: ox.nearest_nodes(G, point.x, point.y))

print(all_shops.head())

              name                    geometry        node
0  Existing Shop 1  POINT (-84.39150 33.77600)   546589748
1  Existing Shop 2  POINT (-84.38850 33.77450)  9244488405
2       New Shop 1  POINT (-84.39200 33.77500)  3158726327
3       New Shop 2  POINT (-84.39000 33.77300)  8845774644
4       New Shop 3  POINT (-84.38800 33.77100)  8584512262


In [18]:
# Compute travel distances
distance_matrix = []

for _, sc in student_centroids.iterrows():
    for _, bs in all_shops.iterrows():
        try:
            distance = nx.shortest_path_length(G, sc["node"], bs["node"], weight="length")
        except nx.NetworkXNoPath:
            distance = np.nan  # If no path exists

        distance_matrix.append({
            "Student_Living_Center": sc["name"],
            "Bike_Shop": bs["name"],
            "Distance_meters": distance,
            "Population": sc["population"]
        })

# Convert to DataFrame
distance_df = pd.DataFrame(distance_matrix)

print(distance_df)

   Student_Living_Center        Bike_Shop  Distance_meters  Population
0                 Dorm A  Existing Shop 1          573.652         500
1                 Dorm A  Existing Shop 2          983.472         500
2                 Dorm A       New Shop 1          450.112         500
3                 Dorm A       New Shop 2          730.905         500
4                 Dorm A       New Shop 3          889.484         500
5                 Dorm B  Existing Shop 1          607.806         300
6                 Dorm B  Existing Shop 2         1035.151         300
7                 Dorm B       New Shop 1          417.647         300
8                 Dorm B       New Shop 2          508.204         300
9                 Dorm B       New Shop 3          697.020         300
10           Apartment C  Existing Shop 1          496.721         700
11           Apartment C  Existing Shop 2          994.387         700
12           Apartment C       New Shop 1          321.192         700
13    

In [21]:
# Huff Model Calculation
b = 1.5  # Distance decay exponent
all_shops["Huff_Attractiveness"] = 1  # Assuming all shops have equal attractiveness

print(all_shops)

              name                    geometry        node  \
0  Existing Shop 1  POINT (-84.39150 33.77600)   546589748   
1  Existing Shop 2  POINT (-84.38850 33.77450)  9244488405   
2       New Shop 1  POINT (-84.39200 33.77500)  3158726327   
3       New Shop 2  POINT (-84.39000 33.77300)  8845774644   
4       New Shop 3  POINT (-84.38800 33.77100)  8584512262   

   Huff_Attractiveness  
0                    1  
1                    1  
2                    1  
3                    1  
4                    1  


In [ ]:
# Compute the probability that students choose each shop
huff_probabilities = []

for _, sc in student_centroids.iterrows():
    student_population = sc["population"]
    relevant_distances = distance_df[distance_df["Student_Living_Center"] == sc["name"]]
    
    print(relevant_distances)

    # Calculate denominator PROBLEM HERE (sum of attractiveness/distance)
    denominator = sum(all_shops.set_index("name").loc[relevant_distances["Bike_Shop"], "Huff_Attractiveness"] / (relevant_distances["Distance_meters"] ** b))
    
    print(denominator)
    
    for _, row in relevant_distances.iterrows():
        bike_shop = row["Bike_Shop"]
        distance = row["Distance_meters"]
        
        # Huff Probability Calculation
        numerator = all_shops.set_index("name").loc[bike_shop, "Huff_Attractiveness"] / (distance ** b)
        probability = numerator / denominator if denominator > 0 else 0
        
        # Store probability weighted by student population
        huff_probabilities.append({
            "Student_Living_Center": sc["name"],
            "Bike_Shop": bike_shop,
            "Probability": probability,
            "Expected_Visits": probability * student_population
        })

# Convert Huff Model results to DataFrame
huff_df = pd.DataFrame(huff_probabilities)

print(huff_df)

  Student_Living_Center        Bike_Shop  Distance_meters  Population
0                Dorm A  Existing Shop 1          573.652         500
1                Dorm A  Existing Shop 2          983.472         500
2                Dorm A       New Shop 1          450.112         500
3                Dorm A       New Shop 2          730.905         500
4                Dorm A       New Shop 3          889.484         500
nan
  Student_Living_Center        Bike_Shop  Distance_meters  Population
5                Dorm B  Existing Shop 1          607.806         300
6                Dorm B  Existing Shop 2         1035.151         300
7                Dorm B       New Shop 1          417.647         300
8                Dorm B       New Shop 2          508.204         300
9                Dorm B       New Shop 3          697.020         300
nan


In [ ]:
# Sum expected visits per shop
expected_demand = huff_df.groupby("Bike_Shop")["Expected_Visits"].sum().reset_index()

# Separate new shops and rank them
new_shop_demand = expected_demand[expected_demand["Bike_Shop"].isin(new_shops["name"])]
new_shop_demand = new_shop_demand.sort_values(by="Expected_Visits", ascending=False)

# Show the best new shop location
print("Best New Bike Shop Location Based on Huff Model:")
print(new_shop_demand)

Best New Bike Shop Location Based on Huff Model:
    Bike_Shop  Expected_Visits
2  New Shop 1                0
3  New Shop 2                0
4  New Shop 3                0
